In [15]:
%run graphika_trending.ipynb
%run misc_functions.ipynb
%run tweepy_tweets_getter.ipynb

In [ ]:
def merge_tags(df):
    global all_tags
    temp_tags = all_tags.reset_index()[['unique_id','tag']].set_index('unique_id')
    df['map_id'] = df['map_id'].astype(str)
    df['cluster_id'] = df['cluster_id'].astype(str)
    df['unique_id'] = df['map_id'].str.cat(df['cluster_id'],sep=".")
    df['unique_id'] = df.unique_id.astype(str)
    temp_tags.index = temp_tags.index.astype(str)
    week_tags = df.reset_index().merge(temp_tags,right_on = 'unique_id',left_on = 'unique_id')
    week_tags = week_tags.set_index('hit_value')
    return week_tags

In [17]:
def get_tagwise_tweets(df):
    for tag in df.tag.unique().tolist():
        display(HTML('<span style="color:black"><h3><u>{}</u></h3></span>'.format(tag)))
        top10 = df.reset_index().message_id.value_counts().index.values.tolist()[:10]
        temp = get_full_tweets(top10)
        tweet_links = []
        for row in temp.iterrows():
            row = row[1]
            tweet_links.append('<a href="https://twitter.com/{1}/status/{0}">{0}</a>'.format(row['tweet_id'],row['screen_name']))
        temp['tweet'] = tweet_links
        temp = temp.set_index('tweet')
        temp = temp.style.format({'tweet': dummy_returner})
#         temp = temp.set_index('tweet_id')
        display(temp)
    #     for message in top10:
    #         try:
    #             display(df[df.message_id == message].set_index('message_id'))
    #         except:
    #             print('No output')
    #             continue
#         break

In [ ]:
### Function to backfill tags

# # gates_noncomm.tag.value_counts()
# # gates_noncomm.node_id.value_counts()
# l = pd.Series
# unknowns = gates_noncomm[gates_noncomm.tag == 'Unknown'].node_id.unique()
# knowns = new_tagged[new_tagged.tag != 'Unknown'].drop_duplicates('node_id')
# for node in unknowns:
# #         l = pd.Series.append(l,pd.Series(knowns[knowns.node_id == node].tag.values.tolist()[0]))
#     try:
#         gates_noncomm[gates_noncomm.node_id == node] = gates_noncomm[gates_noncomm.node_id == node].assign(tag = knowns[knowns.node_id == node].tag.values.tolist()[0])
#     except:
#         pass
# #     break

In [ ]:
def get_tagwise_top_x(df,x = 10):
    df = df[df.map_id_count == 1]
    for tag in df.tag.unique().tolist():
        display(HTML('<span style="color:black"><h3><u>{}</u></h3></span>'.format(tag)))
        try:
            display(get_top_x_trends(df[df.tag == tag],x))
        except:
            print('No output')
            continue

In [ ]:
#Get account creation dates

def get_account_creation_dates(node_list):
    res_dict = {}
    for node in node_list:
        node = str(node)
        url = 'https://api.graphika.com/nodes/{}'.format(node)
        r = re.get(url,auth = (username,pswd))
        response = r.json()['members']
        try:
            map_id = list(response.keys())[-1]
        except:
            pass
        try:
            res_dict[node] = get_node_metadata(node,map_id).account_created_at.values.tolist()
        except:
            res_dict[node] = 'Unknown'
    return res_dict

In [ ]:
#Keeps only one version of a hit if it appears across multiple maps

def drop_duplicate_hits(df):
    return df.reset_index().drop_duplicates(subset = ['hit_value','message_id']).set_index('hit_value')

In [ ]:
def check_where_influence_is(influencer_list,df):
    for node in influencer_list:
        try:
            name = api.get_user(node)._json['name']
            screen_name = api.get_user(node)._json['screen_name']
        except:
            name = 'Unknown'
            screen_name = '@Unknown'
        display(HTML('<span style="color:black">{} {}(@{}) is a top account active in:</span>'.format(node,name,screen_name)))
        display(pd.DataFrame(df[df.node_id == node].tag.value_counts()).rename(columns = {'tag':'count'}).transpose())
#         print('{} {}(@{}) is an a top account'.format(node,name,screen_name))
        url = 'https://api.graphika.com/nodes/{}'.format(node)
        r = re.get(url,auth = (username,pswd))
        response = r.json()['influencers']
        for k in response:
            map_id = k
            cluster_id = str(response[k])
#             print('AKA',get_node_metadata(node,map_id).username.values.tolist()[0])
            try:
                print(all_tags.loc[map_id,cluster_id])
            except:
                url = 'https://api.graphika.com/maps/{}/clusters/{}'.format(map_id,cluster_id)
                r = re.get(url,auth = (username,pswd))
                print('\t-->',r.json()['name'])

In [ ]:
#Fill hits dataframe with full tweets

def fill_trend_data(df,debug = False):
    df = df.reset_index()
    id_list = df.message_id.unique().tolist()
    if debug:
        debug = 1000
    else:
        debug = len(id_list)
    temp = get_full_tweets(id_list[:debug])
    new_df = temp.merge(df.iloc[:debug],left_on = 'tweet_id',right_on = 'message_id')
    return new_df.set_index('hit_value')

In [ ]:
#Get the links that are being used with a given table of hits

def get_associated_links(df,top_x = 5):
    df = df.reset_index()
    url_df = df.explode('urls').dropna()
    res_dict = {'URL':[],'nodes':[],'tweets':[]}
    top_urls = url_df.urls.value_counts().index.values.tolist()[:100]
    for url in top_urls:
        res_dict['URL'].append('<a href="{0}">{0}</a>'.format(url))
        res_dict['nodes'].append(url_df[url_df.urls.str.contains(url)].node_id.nunique())
        res_dict['tweets'].append(url_df[url_df.urls.str.contains(url)].message_id.nunique())
    temp = pd.DataFrame.from_dict(res_dict).set_index('URL').sort_values('tweets',ascending = False).head(top_x)
    temp = temp.style.format({'URL':dummy_returner})
    return temp

In [ ]:
#Return all RTs from a dataframe 

def get_rt_from_df(df,an_id):
    return df[df.retweet_tweet_id == an_id]

In [ ]:
#Print the top hits by each tag
def get_tagwise_hits(df):
    top5 = df.tag.value_counts().index.values.tolist()[:10]
    for t in top5:
        display(HTML('<span style="color:black"><h3><u>{}</u></h3></span>'.format(t)))
        display(pd.DataFrame(df[df.tag == t].index.value_counts().head(5)))

In [ ]:
#Print the text of all tweets under a given tag

# final_result[final_result.screen_name.str.contains('LeboMashia')]
# final_result[final_result.retweet == True].retweet_tweet_id.value_counts()
def get_tagwise_text(df):
    top5 = df.tag.value_counts().index.values.tolist()[:10]
    for t in top5:
        temp_tag_df = df[df.tag == t]
        display(HTML('<span style="color:black"><h3><u>{}</u></h3></span>'.format(t)))
        top5txt = temp_tag_df.retweet_tweet_id.value_counts().index.values.tolist()[:5]
#         display(df[df.tag == t].retweet_tweet_id.value_counts())
        top_tweets = pd.DataFrame()
        for an_id in top5txt:
#             top_tweets.append(temp_tag_df[temp_tag_df.retweet_tweet_id == an_id].text.unique())
            top_tweets = top_tweets.append(temp_tag_df[temp_tag_df.retweet_tweet_id == an_id]).drop_duplicates('retweet_tweet_id')
        count = 0
        for row in top_tweets.iterrows():
            row = row[1]
            color = 'black'
#             count = count + 1
#             if count%2 == 1:
#                 color = 'black'
#             else:
#                 color = 'grey'
#             print(',txt[0],'\n')

            rt_count = temp_tag_df[temp_tag_df.text == row['text']].shape[0]
            display(HTML('<span style="color:{}">--> {} retweet(s) in tag {}:</span>'.format(color,rt_count,t)))
            display(pd.DataFrame(df[df.text == row['text']].tag.value_counts()).rename(columns = {'tag':'count'}).transpose())
            lnk = 'https://twitter.com/{}/status/{}'.format(row['screen_name'],row['tweet_id'])
            display(HTML('<span style="color:{}"><a href="{}">{}</a></span>'.format(color,lnk,row['text'])))
            
            

In [ ]:
#Function to call all other functions

def analyze_hit(hits_df):
    if input('>> Do you want to drop Commercial hits for this analysis? (y/n) \n') == 'y':
        hits_df = hits_df[hits_df.tag != 'Commercial']
    if input('>> Do you want to drop duplicate hits for this analysis? (y/n) \n') == 'y':
        #Drop all hits with more than one instance
#         hits_df = drop_duplicate_hits(hits_df)
        #Drop all hits that feature in more than one map
        hits_df = hits_df[hits_df.map_id_count == 1]
    keep_asking = 'x'
    while(keep_asking != 'y'):
        hit_val = input('>> Enter hit to search and analyze: ')
        temp = hits_df[hits_df.index.str.contains(hit_val,case = False)]
        keep_asking = input('>> Found {} hits. To continue with this, enter (y), otherwise hit Enter and try a different search: '.format(temp.shape[0]))
    hits_df = temp
    tag_choice = input('>> Do you want to narrow down the analysis to a single tag? If so, enter one of the following:\n{} \nElse, hit Enter: '.format(hits_df.tag.unique().tolist()))
    if tag_choice != '':
        hits_df = hits_df[hits_df.tag.str.contains(tag_choice,case=False)]
        print('...Found {} hits.'.format(hits_df.shape[0]))
    hits_df = fill_trend_data(hits_df)
#     display(hits_df)
#     if input('>> Do you want to save these results to a CSV? (y/n) \n') == 'y':
#         print_csv(hits_df)
#     print('...The first 3 tweets in this collection are: ')
    display(HTML('<span style="color:black"><h2>{}</h2></span>'.format('...The first 3 tweets in this collection are: ')))
    display(hits_df.sort_values('hit_time').head(3))

#     print('\n...Getting top URLs associated with this hit')
    display(HTML('<span style="color:black"><h2>{}</h2></span>'.format('...Getting top URLs associated with this hit')))
    assoc_urls = get_associated_links(hits_df)
    display(assoc_urls)
#     print('\n...Checking for influence of five most active nodes')
    display(HTML('<span style="color:black"><h2>{}</h2></span>'.format('...Checking for influence of 5 most active nodes')))
    check_where_influence_is(hits_df.node_id.value_counts().head(5).index.tolist(),hits_df)
#     print('\n...The hits for this search break down as follows across Graphika\'s tags: ')
    display(HTML('<span style="color:black"><h2>{}</h2></span>'.format('...The hits for this search break down as follows across Graphika\'s tags: ')))
    get_tagwise_hits(hits_df)
#     print('\n...The following are the top 5 tweets\' texts for each of Graphika\'s tags: ')
    display(HTML('<span style="color:black"><h2>{}</h2></span>'.format('...The following are the top 5 tweets\' texts for each of Graphika\'s tags: ')))
    get_tagwise_text(hits_df)
    return hits_df